In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import statsmodels.api as sm

In [2]:
datadir = '../data/'

In [3]:
def _read_file(fname, enc='iso8859-8'):
    fd = open(fname, encoding=enc, errors='replace')
    df = pd.read_csv(fd, sep='|')
    return df

def add_model(df):
    df['model'] = df.apply(lambda x: '_'.join([x[y] for y in ['tozeret_cd', 'degem_cd', 'shnat_yitzur', 'sug_degem']]), axis=1)

def get_model_name(ns):
    names = pd.DataFrame(data={'model':ns})
    ret = pd.merge(names, models, how='left', on='model')[['tozeret_nm', 'kinuy_mishari']]
    return ret

In [4]:
# some of the makes are make + country of origin. Map those to just make.
# Also, simplify some of the names

def convert_make(df, oldcol='tozeret_nm', newcol='make'):
    # read dictionary
    filename=datadir + 'makes_dict.csv'

    with open(filename, 'r') as f:
        lines = f.readlines()

    make_dict = []
    for line in lines:
        line = line.strip()
        items = line.split(',', 1)
        itm = items[0].strip()
        if len(items) > 1:
            make_dict.append((itm, items[1].strip()))
        else:
            make_dict.append((itm, itm))
    
    newdat = df[oldcol].copy()
    for (m_in, m_out) in make_dict:
        newdat[newdat.str.startswith(m_in)] = m_out
    df[newcol] = newdat

In [5]:
# Figure if a vehicle was 1. never traded 2. traded within 12 months of purchase 3. traded later
def trade_category(d):
    oc = d['ownership_count']
    months = d['months_to_first_trade']
    if math.isnan(oc):
        return 'ללא'
    if (oc == 1) & (months == 0):
        return 'מקורי'
    if months < 12:
        return 'נמכר תוך שנה'
    return 'נמכר תוך יותר משנה'

In [6]:
def read_file(fname):
    fname = datadir + fname
    df = _read_file(fname)

    for c in ['mispar_rechev', 'degem_cd', 'tozeret_cd', 'shnat_yitzur']:
        df[c] = df[c].astype(str)

    df['test']= pd.to_datetime(df.mivchan_acharon_dt)
    df['test_expiry']= pd.to_datetime(df.tokef_dt)
    df['year'] = df['shnat_yitzur'].astype(int)
    c='moed_aliya_lakvish'
    df[c] = pd.to_datetime(df[c], format="%Y-%m")
    df['kvish_ym'] = df[c].dt.strftime('%Y%m')
    #df['sidra'] = df['mispar_rechev'].apply(lambda k : k[-2:])
    add_model(df)
    convert_make(df)
    return df

In [7]:
vo = pd.read_csv(datadir + 'vehicle_ownership.csv', sep='|')

c='mispar_rechev'
vo[c] = vo[c].astype(str)

c='baalut_dt'
vo[c] = pd.to_datetime(vo[c], format="%Y%m")

In [8]:
vo.baalut.value_counts()

פרטי     1619006
החכר      597239
סוחר      393190
חברה      141107
השכרה      38370
Name: baalut, dtype: int64

In [9]:
# count number of ownerships
oc = vo.groupby('mispar_rechev').size()
just1 = oc[oc == 1]
just1.name='ownership_count'
notjust1 = oc[~(oc == 1)]
notjust1.name='ownership_count_'

In [10]:
# The first ownership date is the purchase date. Pick the second-earliest date.
# This takes a long time, so we just do it for vehicles with at least two ownership dates.
mult_own = vo.merge(notjust1, how='right', left_on='mispar_rechev', right_index=True)

nownerships = mult_own.groupby('mispar_rechev').agg({'baalut_dt' : lambda x: x.nsmallest(2).iloc[-1], 'baalut' : 'count'})
nownerships.rename(columns={'baalut' : 'ownership_count'}, inplace=True)
nownerships.reset_index(inplace=True)

In [11]:
# add the vehicles with just one ownership date
single_own = vo.merge(just1, how='right', left_on='mispar_rechev', right_index=True).drop(columns=['baalut'])

nownerships = pd.concat([nownerships, single_own])

In [12]:
# merge with the master file with all vehicles
# source: https://data.gov.il/dataset/shinui_mivne

df = read_file('rechev.csv')

In [13]:
df = pd.merge(df, nownerships, on='mispar_rechev', how='left')

In [14]:
df_bak = df.copy()
#df = df.dropna(subset=['baalut_dt'])

In [15]:
revert = False
if revert:
    df = df_bak.copy()
    revert = False

In [16]:
# Remove the commercial (MISHARI) models. There is very little ownership data on those anyway
df.drop(index=df[df['sug_degem'] == 'M'].index, inplace=True)

In [17]:
# compute number of days and months till first trade
c = 'days_to_first_trade'
df[c] = ((df['baalut_dt'] - df['moed_aliya_lakvish']).dt.total_seconds())/(24*60*60)
c2 = 'months_to_first_trade'
df[c2] = df[c]/30

df['trade'] = df.apply(trade_category, axis=1)

In [18]:
# there is a small number of mistyped ownership dates which precede the initial date, drop them
df.drop(index=df[df['days_to_first_trade'] < 0].index, inplace=True)

In [19]:
traded = df.query('ownership_count > 1')

if False:
    cols = ['mispar_rechev', 'sug_degem', 'tozeret_nm',
       'shnat_yitzur', 'mivchan_acharon_dt', 'tokef_dt',
       'baalut', 'sug_delek_nm', 'moed_aliya_lakvish',
       'kinuy_mishari', 'test', 'test_expiry', 'model', 'baalut_dt',
       'ownership_count', 'days_to_first_trade']
    traded[cols].to_csv(datadir + 'traded.csv', index=False)

In [20]:
if False:
    d=traded.query("year <= 2020")
    print(d['sug_delek_nm'].value_counts())
    sns.violinplot(data=d, x='sug_delek_nm', y='days_to_first_trade')
    plt.gca().set_xticklabels([tick.get_text()[::-1] for tick in plt.gca().get_xticklabels()])

    d=traded.query("year > 2020")
    print(d['sug_delek_nm'].value_counts())
    sns.violinplot(data=d, x='sug_delek_nm', y='days_to_first_trade')
    # Set the reversed tick labels
    plt.gca().set_xticklabels([tick.get_text()[::-1] for tick in plt.gca().get_xticklabels()])

In [57]:
def plot_deals(df, groupby='sug_delek_nm', title='התפלגות זמן מכירה לפי סוג דלק'):
    D='months_to_first_trade'
    df_filtered = df[~df['sug_delek_nm'].isin(low_fuel_counts)]

    groups = df_filtered.groupby(groupby)
    fig = go.Figure()
    for name, group in groups:
        ecdf = sm.distributions.ECDF(group[D])
        x = np.linspace(min(group[D]), max(group[D]), num=100)
        y = ecdf(x)
        fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=name))

    # customize the layout
    fig.update_traces(line=dict(width=3))
    fig.update_layout(title=title, xaxis_title='משך זמן מהקניה בחודשים', yaxis_title='הסתברות מכירה עד תום תקופה')

    # show the plot
    fig.show()

In [22]:
#yearfuel = df.query("year >= 2020").groupby(['kvish_ym', 'sug_delek_nm']).size().reset_index().rename(columns={0 : "count"})

In [39]:
# Count market share over time, by fuel type
counts=df.query("year >= 2020").groupby('kvish_ym')['sug_delek_nm'].value_counts(normalize=True)
counts.name='share'
counts = counts.reset_index()

,kvish_ym,sug_delek_nm,share
0,202001,בנזין,0.853571
1,202001,דיזל,0.100762
2,202001,חשמל/בנזין,0.034895
3,202001,חשמל,0.009844
4,202001,"גפמ""",0.000813
...,...,...,...
229,202304,בנזין,0.707582
230,202304,חשמל,0.171030
231,202304,חשמל/בנזין,0.071941
232,202304,דיזל,0.048866


In [56]:
# Filter out rows with little data
fuel_counts=df['sug_delek_nm'].value_counts()
low_fuel_counts = fuel_counts[fuel_counts < 50000].index.to_list()

df_filtered = counts[~counts['sug_delek_nm'].isin(low_fuel_counts)]

fig = px.line(df_filtered, x='kvish_ym', y='share', color='sug_delek_nm', line_shape='spline')
fig.update_traces(line=dict(width=3))

fig.update_layout(xaxis_tickangle=45)

fig.show()

In [25]:
df['trade'].value_counts()

ללא                   1951729
מקורי                  982802
נמכר תוך יותר משנה     422670
נמכר תוך שנה           199013
Name: trade, dtype: int64

In [26]:
traded = df.query('ownership_count > 1')

traded['trade'].value_counts()

נמכר תוך יותר משנה    422459
נמכר תוך שנה          198985
Name: trade, dtype: int64

In [27]:
counts = traded.query('year >= 2020').groupby('sug_delek_nm')['trade'].value_counts(normalize=True)
counts.name='share'
counts = counts.reset_index()

df_filtered = counts[~counts['sug_delek_nm'].isin(low_fuel_counts)]

fig = px.bar(df_filtered, x="sug_delek_nm", y="share", color="trade", barmode="stack")

fig.show()

In [28]:
counts = df.query('year >= 2020').groupby('sug_delek_nm')['trade'].value_counts(normalize=True)
counts.name='share'
counts = counts.reset_index()

df_filtered = counts[~counts['sug_delek_nm'].isin(low_fuel_counts)]

fig = px.bar(df_filtered, x="sug_delek_nm", y="share", color="trade", barmode="stack")

fig.show()

In [58]:
plot_deals(traded)

In [59]:
plot_deals(traded.query("year < 2020"), title='התפלגות זמן מכירה לפי סוג דלק, שנה < 2020')

In [60]:
plot_deals(traded.query("year >= 2020"), title='התפלגות זמן מכירה לפי סוג דלק, שנה >= 2020')

In [61]:
big_makes = df.query("sug_delek_nm == 'חשמל' and year >= 2020")['make'].value_counts().head(8)
big_makes['Aiways'] = 10
traded_big = traded.merge(big_makes, how='right', left_on='make', right_index=True)

plot_deals(traded_big.query("sug_delek_nm == 'חשמל' and year >= 2020"), groupby='make', title='Months to trade by make, electric year >= 2020')

In [62]:
fuel = 'דיזל'
fuel = 'בנזין'
big_makes = df.query("sug_delek_nm == @fuel and year >= 2020")['make'].value_counts().head(8)
traded_big = traded.merge(big_makes, how='right', left_on='make', right_index=True)

plot_deals(traded_big.query("sug_delek_nm == @fuel and year >= 2020"), groupby='make', title='Months to trade by make, ' + fuel +' year >= 2020')

In [34]:
big_makes

יונדאי      131416
טויוטה      111824
קיה          93971
מזדה         51116
סקודה        49445
מיצובישי     36342
סיאט         34508
סוזוקי       32255
Name: make, dtype: int64

In [35]:
from datetime import datetime

now = datetime.now()
df['days_on_road'] = ((df['test'] - df['moed_aliya_lakvish']).dt.total_seconds())/(24*60*60)
df['ownership_days'] = ((now - df['moed_aliya_lakvish']).dt.total_seconds())/(24*60*60)

In [36]:
df['ownership_count_'] = df['ownership_count'].fillna(1)

In [37]:
agg_own = df.query('year == 2020')[['ownership_days', 'ownership_count_', 'make']].groupby('make').sum()
agg_own['avg_own_days'] = agg_own['ownership_days'] / agg_own['ownership_count_']
agg_own.query("ownership_count_ > 1000").sort_values(by='avg_own_days')

,ownership_days,ownership_count_,avg_own_days
make,,,
BMW,2.893320e+06,6353.0,455.425828
רובר,1.079093e+06,2084.0,517.798755
מרצדס,4.147593e+06,7870.0,527.013091
אאודי,3.474541e+06,6392.0,543.576457
קרייזלר,1.663988e+06,3050.0,545.569778
מזדה,1.015725e+07,16373.0,620.366053
רנו,7.580012e+06,10978.0,690.472940
ניסאן,8.133175e+06,11658.0,697.647564
לקסוס,1.585771e+06,2256.0,702.912815
